In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Approach :** 

In  this note book i try to explor via SHAP the correlation beetween districts features,and the percentage of students that make access on the palforms. To do this i use the mean pct_access of the 5 most frequently used paltforms in their district. 

In order to do this i take a dive into each district that does not contain any Na stats. No districts with Na's are considered. This is just a first approach, a sort of EDA to get a feel of the data. 


I leave the following data feature tags below from the EDA notebooks because of the importance of understanding what the stats are if they are to be analysed in a social demographic context.

DISTRICT DATA overview:

district_id - The unique identifier of the school district

state - The state where the district resides in

locale - NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See Locale Boundaries User's Manual for more information.

pct_black/hispanic - Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data

pct_free/reduced - Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data

countyconnectionsratio - ratio that for reasons below i decide not to use

pptotalraw - Per-pupil total expenditure (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district.

In [ ]:
df1 = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv", low_memory=False)
print(df1.shape)
df1.head()

county_connections_rati nearly all is 0.18,-1  or Nan, 
the 4 district with Nan as connection seem to disurupt the model, may be interesting to see why; But for Now as there is very little varaince among the districts 
we have got data on we shall discard this column. 

In [ ]:
print(df1.county_connections_ratio.unique())
print(np.mean([df1.county_connections_ratio== '[0.18, 1[']))
df1_2=df1.dropna()
nnadf=df1_2.drop(labels=('county_connections_ratio'),axis=1)
dists_wdata=nnadf.district_id.values

In [ ]:
nnadf['pct_black/hispanic'].describe()

In [ ]:
nnadf['pct_black/hispanic'].value_counts()

ENGAGEMENT DATA per District overview:

each district has its own file that holds the following data:

time - date in "YYYY-MM-DD"

lp_id - The unique identifier of the product

pct_access - Percentage of students in the district have at least one page-load event of a given product and on a given day

engagement_index - Total page-load events per one thousand students of a given product and on a given day

So from this file..

In [ ]:
df = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data/8815.csv", low_memory=False)
print(df.shape)
df.head()

i get the top 5 platforms that scored the highest overall mean pct_access in that district.

In [ ]:
df.groupby('lp_id').mean().sort_values('pct_access',ascending=False).head(5)

I will afterwards do the mean over them also. This i s very crude, a fast analysis to see what seems to amtter the most about the districts. 


In order to unite all the files quickly i resort to nummpy witch is mutch faster for this than directly in pandas. 
This is a habit, probably not necessary for sutch a small resulting data set. For the Random forest model i need to move away from the object type the columns were in into a numerical rappresentation of the categories so i therefor create the dictionary used for the mapping 

In [ ]:
#Create Dictionary For the Zones
i=0
zones = {}
for zone in nnadf.locale.unique():
    zones[zone]=i
    i+=1
#zones={'Rural':0, 'Town': 1,'Suburb': 2, 'City': 3}
print('Area zone category tags:',zones)
#Make the categorical Data
nnadf.locale=nnadf.locale.map(zones)

#Create Dictionary For the pp_total
i=0
pp_ = {}
for pp in nnadf.pp_total_raw.unique():
    pp_[pp]=i
    i+=1
    
pp_={'[4000, 6000[':0, '[6000, 8000[':1,'[8000, 10000[':2,'[10000, 12000[':3,'[12000, 14000[':4,'[14000, 16000[':5,
   '[16000, 18000[':6,'[18000, 20000[':7,'[22000, 24000[':8,'[32000, 34000[':9}
print('pp_total bracket cat:',pp_)
nnadf.pp_total_raw=nnadf.pp_total_raw.map(pp_)


#Create Dictionary For the pct_black/hisp
i=0
p_bh = {}
for p in nnadf['pct_black/hispanic'].unique():
    p_bh[p]=i
    i+=1
p_bh = {'[0, 0.2[': 0, '[0.2, 0.4[': 1, '[0.4, 0.6[': 2, '[0.6, 0.8[': 3,'[0.8, 1[' : 4}
print('pct_black/hispanic bracket category:',p_bh)
nnadf['pct_black/hispanic']=nnadf['pct_black/hispanic'].map(p_bh)


#Create Dictionary For the pct_free/reduced
i=0
p_fr = {}
for p in nnadf['pct_free/reduced'].unique():
    p_fr[p]=i
    i+=1
print('pct_free/reduced bracket category :',p_fr)
nnadf['pct_free/reduced']=nnadf['pct_free/reduced'].map(p_fr)






# Here is the connected data 
between the two datasets ( district features file and the access files of each district)

In [ ]:
nnadf=nnadf.drop(labels=('state'),axis=1)
nnadf=nnadf.set_index('district_id')
Districts_data=np.empty(shape=(dists_wdata.shape[0],10))
i=0
for dist in dists_wdata:      
    d=str(dist)
    df = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data/{}.csv".format(d), low_memory=False)
    dist_top5_Ip= df.groupby('lp_id').mean().sort_values('pct_access',ascending=False).head(5)
    #Districts_data[i,[0]]=dist
    Districts_data[i,0]=dist#dsitrict name
    Districts_data[i,1:5]=nnadf.loc[[dist]].values#district data
    Districts_data[i,5]=dist_top5_Ip.pct_access.mean() # mean of the mean of the top 5 most accessed ip ( platforms)
    Districts_data[i,6]=dist_top5_Ip.engagement_index.mean()
    Districts_data[i,7]=dist_top5_Ip.index[0] #districts most used paltform 
    Districts_data[i,8]=dist_top5_Ip.index[1]#districts second most used paltform 
    Districts_data[i,9]=dist_top5_Ip.index[2]# ...thrid...
    i+=1
    
data= pd.DataFrame(Districts_data)
data=data.dropna()
data.columns=['district','locale', 'pct_black/hispanic', 'pct_free/reduced',
        'pp_total_raw','most_usedip_pct_access_mean','most_usedip_eng_mean','most_accessed_ip','second_m_a_ip','third_m_a_ip']
data.head()

**With the Data ready let's start with the SHAP !**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


y = data['most_usedip_pct_access_mean']  
feature_names=['locale', 'pct_black/hispanic', 'pct_free/reduced',
        'pp_total_raw','most_accessed_ip']
X = data[feature_names]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
my_model = RandomForestRegressor(random_state=0,max_leaf_nodes=10,min_samples_leaf=5 ).fit(train_X, train_y)

In [ ]:
#Quick test on the Val data
row_to_show = 3
data_for_prediction = val_X.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)
print('pred:',my_model.predict(data_for_prediction_array),'True:',val_y.iloc[row_to_show])

# **Feature importance**

In [ ]:
#Feature improtances
dict(zip(my_model.feature_importances_,feature_names))

Good to see 'pct_black/hispanic' bearing such little weight !! Hence the title
having said that, one must remember waht it is we are modelling. It is the access probability of each student from a given district. Now the model shows it does not matter if the district has a higher population of people with black or Hispanic Heritage, the probability that any given student will acess will reamain the same. It does not seem to play a role.

to confirm that this feature in the selected dataset (districts without Nans in data) is not represented i do an quick analysis of it's composition vs the original data of all the districts(df1)
thow the rappresentation is half, the distriution seems to hold perfectly fine,so no issue there.

In [ ]:
df1['pct_black/hispanic'].value_counts(),nnadf['pct_black/hispanic'].value_counts()

In [ ]:
#see if  overfitting
print(my_model.score(train_X,train_y),my_model.score(val_X,val_y))
# Comapre r2 score VS a stupid mean model  
from sklearn.metrics import r2_score
val_y_pred= my_model.predict(val_X)
val_y_mean= [train_y.mean() for x in val_y_pred]
r2_score(val_y, val_y_pred),r2_score(val_y, val_y_mean)

We are not overfitting and the model as approximative as it is doing significantly better than simply guessing the mean !!

In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(my_model)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction)

# Shap Plot & Deductions

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values, data_for_prediction)

Remebering the Mapping :

Area zone category tags: {'Suburb': 0, 'Rural': 1, 'City': 2, 'Town': 3}
pp_total bracket cat: {'[4000, 6000[': 0, '[6000, 8000[': 1, '[8000, 10000[': 2, '[10000, 12000[': 3, '[12000, 14000[': 4, '[14000, 16000[': 5, '[16000, 18000[': 6, '[18000, 20000[': 7, '[22000, 24000[': 8, '[32000, 34000[': 9}
pct_black/hispanic bracket category: {'[0, 0.2[': 0, '[0.2, 0.4[': 1, '[0.4, 0.6[': 2, '[0.6, 0.8[': 3, '[0.8, 1[': 4}
pct_free/reduced bracket category : {'[0, 0.2[': 0, '[0.2, 0.4[': 1, '[0.4, 0.6[': 2, '[0.6, 0.8[': 3, '[0.8, 1[': 4}

**DEDUCTIONS :**

This district scored higher beacuse of the hgiher amount of pubblic funding per Student (high pp_total) and because it was in the suburbs, Less beacuse it was a subsided area ( payign students more likly to log in i guess) 
Another infleunecer was the mostused platform in the district that could be may be less user friendly and attractive. 

That's it folks !

this was written with the best intentions,
Any comments, signs of appretiation, doubts and suggestions are more than welcome, thank you !